# Camera Acquisition Application

This notebook implements a camera acquisition app using `PySpin` for camera control, `OpenCV` for image processing, and `Tkinter` for the GUI. It supports real-time video capture, annotation, and saving of raw video data, along with thresholding and blob detection for real-time object tracking.

### Imports
- **`PySpin`**: Used for controlling the camera hardware and configuring its settings.
- **`cv2` (OpenCV)**: Used for image processing (thresholding, contour finding, and drawing) and video saving.
- **`socket`**: Handles UDP communication for sending the tracked object coordinates to another system in real-time.
- **`numpy`**: Provides array manipulation and other numerical operations.
- **`csv`**: Used for logging data, such as object positions and timestamps, into CSV files for further analysis.
- **`tkinter`**: Provides the graphical user interface (GUI) for camera settings and controlling video acquisition.

### Overview
The application provides a GUI for configuring camera settings such as frame rate, exposure time, output filename, and threshold values. Once the "Start Acquisition" button is pressed, the application:
1. Configures the camera and begins video acquisition.
2. Processes the video feed in real-time, applying binary thresholding to detect objects.
3. Tracks the largest object, calculates its center position, and logs its coordinates (in millimeters) along with the timestamp to a CSV file.
4. Optionally saves the video feed in `.avi` format.
5. Sends the position data to a remote system using UDP.

### Camera Configuration
The camera is configured using `PySpin` to:
- Set the acquisition mode to continuous.
- Set the pixel format to Mono8 (grayscale).
- Configure the maximum width and height of the video frames.
- Enable frame rate control and set the desired frame rate.
- Disable auto-exposure and auto-gain, allowing for manual control of the exposure time and gain.
  
### Video Acquisition
In real-time:
1. The video frames are processed by applying binary thresholding using the user-defined threshold values.
2. Contours are found in the binary mask, and the largest contour is tracked.
3. The center of the largest contour is computed and its position is converted from pixels to millimeters based on a predefined conversion factor.
4. The data (position and timestamp) is logged to a CSV file and sent over UDP.
5. Optionally, the video frames are saved to an `.avi` file.

### Real-Time Object Tracking
The application uses **OpenCV** to:
- Apply thresholding to detect objects in the video feed.
- Find contours of the detected objects and select the largest one.
- Calculate the center of mass of the largest contour and display it on the video feed.

### User Interface
The user interface, implemented with **Tkinter**, includes:
- **FPS and Exposure Controls**: Text entry fields allow the user to specify the frame rate and exposure time.
- **Threshold and Blob Size Controls**: Scales to adjust the minimum and maximum threshold values for object detection and the minimum and maximum blob sizes.
- **Start and Stop Buttons**: The "Start Acquisition" button begins the video capture, while the "Stop Acquisition" button halts the process.
- **Save Video Option**: A checkbox allows the user to choose whether to save the video feed to a file.

### Usage
1. Set the desired frame rate, exposure time, and output filename.
2. Adjust the threshold values and blob size limits to suit the object being tracked.
3. Press "Start Acquisition" to begin capturing and processing the video.
4. The application will display the annotated video feed and save the data to a CSV file.
5. Press "Stop Acquisition" to stop the process.

### UDP Communication
The application uses a socket to send real-time position data (in millimeters) to a remote system via UDP. The coordinates are sent in the format `(y_mm, x_mm)` along with a timestamp.

### Example Workflow:
- The user adjusts camera settings using the GUI.
- The user clicks "Start Acquisition", and the camera begins capturing frames.
- Each frame is processed in real-time, with object tracking, annotation, and logging.
- The user can stop the acquisition by pressing the "Stop Acquisition" button.


In [1]:
import PySpin
import cv2
import socket
import numpy as np
import csv
from datetime import datetime
import tkinter as tk
from tkinter import ttk, filedialog
import threading

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Camera Acquisition App")

        # Variables to store camera settings
        self.fps = tk.IntVar(value=50)
        self.exposure = tk.IntVar(value=5000)
        self.output_filename = tk.StringVar(value='output')
        self.min_threshold_value = tk.IntVar(value=200)
        self.max_threshold_value = tk.IntVar(value=255)
        self.min_blob_size = tk.IntVar(value=0)
        self.max_blob_size = tk.IntVar(value=300)
        self.save_video = tk.BooleanVar(value=True)

        # Create GUI elements
        self.create_widgets()

        # ArUco detector
        self.detector = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)

        # VideoWriter and CSV file variables
        self.out = None
        self.csv_file = None
        self.stop_acquisition = threading.Event()  # Event to signal stop of acquisition

    def create_widgets(self):
        # Frame for camera settings
        settings_frame = ttk.LabelFrame(self.root, text="Camera Settings")
        settings_frame.grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)

        # FPS Entry
        ttk.Label(settings_frame, text="FPS:").grid(row=0, column=0, sticky=tk.W)
        fps_entry = ttk.Entry(settings_frame, textvariable=self.fps)
        fps_entry.grid(row=0, column=1, padx=5, pady=5)

        # Exposure Entry
        ttk.Label(settings_frame, text="Exposure (us):").grid(row=1, column=0, sticky=tk.W)
        exposure_entry = ttk.Entry(settings_frame, textvariable=self.exposure)
        exposure_entry.grid(row=1, column=1, padx=5, pady=5)

        # Output Filename Entry
        ttk.Label(settings_frame, text="Output Filename:").grid(row=2, column=0, sticky=tk.W)
        output_entry = ttk.Entry(settings_frame, textvariable=self.output_filename)
        output_entry.grid(row=2, column=1, padx=5, pady=5)

        # Min Threshold Scale
        ttk.Label(settings_frame, text="Min Threshold:").grid(row=3, column=0, sticky=tk.W)
        min_threshold_scale = tk.Scale(settings_frame, from_=0, to=255, orient=tk.HORIZONTAL, variable=self.min_threshold_value)
        min_threshold_scale.grid(row=3, column=1, padx=5, pady=5)

        # Max Threshold Scale
        ttk.Label(settings_frame, text="Max Threshold:").grid(row=4, column=0, sticky=tk.W)
        max_threshold_scale = tk.Scale(settings_frame, from_=0, to=255, orient=tk.HORIZONTAL, variable=self.max_threshold_value)
        max_threshold_scale.grid(row=4, column=1, padx=5, pady=5)

        # Min Blob Size Scale
        ttk.Label(settings_frame, text="Min Blob Size:").grid(row=5, column=0, sticky=tk.W)
        min_blob_size_scale = tk.Scale(settings_frame, from_=0, to=2000, orient=tk.HORIZONTAL, variable=self.min_blob_size)
        min_blob_size_scale.grid(row=5, column=1, padx=5, pady=5)

        # Max Blob Size Scale
        ttk.Label(settings_frame, text="Max Blob Size:").grid(row=6, column=0, sticky=tk.W)
        max_blob_size_scale = tk.Scale(settings_frame, from_=0, to=2000, orient=tk.HORIZONTAL, variable=self.max_blob_size)
        max_blob_size_scale.grid(row=6, column=1, padx=5, pady=5)

        # Checkbox for video saving
        save_video_checkbox = ttk.Checkbutton(settings_frame, text="Save Video", variable=self.save_video)
        save_video_checkbox.grid(row=7, column=0, columnspan=2, padx=5, pady=5)

        # Start Button
        start_button = ttk.Button(self.root, text="Start Acquisition", command=self.start_acquisition)
        start_button.grid(row=1, column=0, padx=10, pady=10)

        # Stop Button
        stop_button = ttk.Button(self.root, text="Stop Acquisition", command=self.stop_acquisition_process)
        stop_button.grid(row=1, column=1, padx=10, pady=10)

    def browse_output_file(self):
        filename = filedialog.asksaveasfilename(defaultextension=".avi", filetypes=[("AVI files", "*.avi")])
        if filename:
            self.output_filename.set(filename)

    def configure_camera(self, cam):
        nodemap = cam.GetNodeMap()
        
        # Set acquisition mode to continuous
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())
        
        # Set pixel format to Mono8 (8-bit grayscale)
        node_pixel_format = PySpin.CEnumerationPtr(nodemap.GetNode('PixelFormat'))
        node_pixel_format_mono8 = node_pixel_format.GetEntryByName('Mono8')
        node_pixel_format.SetIntValue(node_pixel_format_mono8.GetValue())
        
        # Set maximum width
        node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
        max_width = node_width.GetMax()
        node_width.SetValue(max_width)
        
        # Set maximum height
        node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
        max_height = node_height.GetMax()
        node_height.SetValue(max_height)
        
        # Enable frame rate control
        node_acquisition_frame_rate_enable = PySpin.CBooleanPtr(nodemap.GetNode('AcquisitionFrameRateEnable'))
        node_acquisition_frame_rate_enable.SetValue(True)

        # Set frame rate
        node_frame_rate = PySpin.CFloatPtr(nodemap.GetNode('AcquisitionFrameRate'))
        node_frame_rate.SetValue(self.fps.get())
        
        # Turn off auto exposure
        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
        node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
        
        # Turn off auto gain
        node_gain_auto = PySpin.CEnumerationPtr(nodemap.GetNode('GainAuto'))
        node_gain_auto_off = node_gain_auto.GetEntryByName('Off')
        node_gain_auto.SetIntValue(node_gain_auto_off.GetValue())
        
        # Set exposure time (in microseconds)
        node_exposure_time = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        node_exposure_time.SetValue(self.exposure.get())
        
        # Set gain (if needed)
        node_gain = PySpin.CFloatPtr(nodemap.GetNode('Gain'))
        node_gain.SetValue(10.0)  # Example: set gain to 10 dB

    def acquire_video(self, cam):
        # Set up socket for UDP communication
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        serverAddressPort = ("127.0.0.1", 5053)

        # Initialize CSV writer
        self.csv_file = open(f'{self.output_filename.get()}.csv', 'w', newline='')
        csv_writer = csv.writer(self.csv_file)
        csv_writer.writerow(['Timestamp', 'Center_X_mm', 'Center_Y_mm'])

        # Initialize VideoWriter if "Save Video" checkbox is checked
        if self.save_video.get():
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            frame_width = int(cam.Width())
            frame_height = int(cam.Height())
            self.out = cv2.VideoWriter(f'{self.output_filename.get()}.avi', fourcc, self.fps.get(), (frame_width, frame_height), isColor=False)
        else:
            self.out = None

        try:
            # Start the acquisition
            cam.BeginAcquisition()

            while not self.stop_acquisition.is_set():
                # Retrieve next image
                image_result = cam.GetNextImage()

                if image_result.IsIncomplete():
                    print('Image incomplete: ', image_result.GetImageStatus())
                else:
                    # Convert image to numpy array (Grayscale)
                    frame = image_result.GetNDArray()

                    # Create a copy of the frame for annotation
                    annotated_frame = frame.copy()

                    # Apply binary thresholding to create a mask
                    _, mask = cv2.threshold(frame, self.min_threshold_value.get(), self.max_threshold_value.get(), cv2.THRESH_BINARY)

                    # Find contours in the binary mask
                    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    # Check if any contours are found
                    if contours:
                        # Sort contours by area, largest first
                        largest_contour = max(contours, key=cv2.contourArea)

                        # Calculate the area of the largest contour
                        area = cv2.contourArea(largest_contour)

                        # Proceed if the area is within the specified range
                        if self.min_blob_size.get() <= area <= self.max_blob_size.get():
                            M = cv2.moments(largest_contour)
                            if M["m00"] != 0:
                                cX = int(M["m10"] / M["m00"])
                                cY = int(M["m01"] / M["m00"])

                                # Convert pixel coordinates to millimeters from the center
                                x_mm = (cX - frame.shape[1] / 2) * (195 / 1024)
                                y_mm = (frame.shape[0] / 2 - cY) * (195 / 1024)

                                # Get current timestamp
                                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

                                # Write data to CSV file
                                csv_writer.writerow([timestamp, x_mm, y_mm])
                                self.csv_file.flush()  # Ensure data is written immediately

                                data = (y_mm, x_mm)
                                print(data)
                                sock.sendto(str.encode(str(data)), serverAddressPort)

                                # Draw the bounding rectangle and center point on the annotated frame
                                (x, y, w, h) = cv2.boundingRect(largest_contour)
                                cv2.rectangle(annotated_frame, (x, y), (x+w, y+h), (255, 255, 255), 2)  # White rectangle
                                cv2.putText(annotated_frame, f"Area: {int(area)}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    # Display the annotated frame
                    annotated_frame_resized = cv2.resize(annotated_frame, (0, 0), None, 0.5, 0.5)
                    height, width = annotated_frame_resized.shape[:2]
                    center_x, center_y = width // 2, height // 2
                    cv2.line(annotated_frame_resized, (0, center_y), (width, center_y), (255, 255, 255), 1)
                    cv2.line(annotated_frame_resized, (center_x, 0), (center_x, height), (255, 255, 255), 1)
                    cv2.imshow('Video', annotated_frame_resized)

                    # Save the raw frame to the video file
                    if self.out is not None:
                        self.out.write(frame)

                    # Check for 'Esc' key press to exit (optional)
                    if cv2.waitKey(1) == 27:  # Esc key
                        break

                # Release image
                image_result.Release()

        finally:
            # End acquisition
            cam.EndAcquisition()

            # Close CSV file
            self.csv_file.close()

            # Release VideoWriter
            if self.out is not None:
                self.out.release()

            # Close socket
            sock.close()

            # Destroy OpenCV window
            cv2.destroyAllWindows()


    def start_acquisition(self):
        # Start acquisition in a separate thread
        self.stop_acquisition.clear()  # Clear the stop event flag
        acquisition_thread = threading.Thread(target=self.run_acquisition)
        acquisition_thread.start()

    def run_acquisition(self):
        system = PySpin.System.GetInstance()
        cam_list = system.GetCameras()

        try:
            if cam_list.GetSize() == 0:
                raise ValueError("No cameras found!")

            # Assume one camera for simplicity
            cam = cam_list.GetByIndex(0)

            # Initialize camera
            cam.Init()

            # Configure camera settings
            self.configure_camera(cam)

            # Acquire video until stop event is set
            self.acquire_video(cam)

            # Deinitialize camera
            cam.DeInit()

        except PySpin.SpinnakerException as ex:
            print('Error: %s' % ex)
            exit_code = 1

        finally:
            # Release camera and system resources
            del cam
            cam_list.Clear()
            system.ReleaseInstance()

    def stop_acquisition_process(self):
        # Set the stop event flag to stop acquisition
        self.stop_acquisition.set()

if __name__ == '__main__':
    root = tk.Tk()
    app = CameraApp(root)
    root.mainloop()

(-96.9287109375, -87.4072265625)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.4072265625)
(-96.9287109375, -87.4072265625)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.4072265625)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96.9287109375, -87.216796875)
(-96